**Cài đặt thư viện UnderTheSea để hỗ trợ tách từ trong tiếng Việt. Xem thêm các tính năng của thư viện tại: https://github.com/undertheseanlp/underthesea**

In [2]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 4.0 MB/s 
     |████████████████████████████████| 591 kB 7.6 MB/s 
     |████████████████████████████████| 1.0 MB 47.8 MB/s 


**Import các thư viện cần thiết**

In [3]:
import os
import re
import math
import numpy as np
from scipy.spatial import distance
from underthesea import text_normalize
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

**Lấy dữ liệu các tài liệu/văn bản mẫu từ**: https://github.com/HUTECH-OpenCourseWare/IRS.git

In [4]:
!git clone https://github.com/HUTECH-OpenCourseWare/IRS.git

Cloning into 'IRS'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 271 (delta 0), reused 271 (delta 0), pack-reused 0
Receiving objects: 100% (271/271), 441.48 KiB | 16.35 MiB/s, done.


**Tiến hành thử nghiệm với danh sách các tài liệu/văn bản thuộc các chủ đề khác nhau**

In [5]:
# Chọn danh sách các chủ đề của tài liệu/văn bản cho thử nghiệm
topics = [
    'the-thao',
    'du-lich',
    'khoa-hoc'
]

number_of_topics = len(topics)

# Với mỗi chủ đề - chúng ta sẽ giới hạn số lượng tài liệu/văn bản 
# Nhằm đảm bảo đồng đều về số lượng tài liệu/văn bản cho từng chủ đề
limit_doc_per_topic = 5

# Tạo một tập dữ liệu thử nghiệm gồm các tài liệu/văn bản thuộc về 2-3 chủ đề
# Cấu trúc dữ liệu dạng list - lưu thông tin danh sách các tài liệu/văn bản thuộc chủ đề khác nhau
# Mỗi tài liệu/văn bản sẽ tổ chức dạng 1 tuple với: (topic, nội_dung_văn_bản, danh_sách_token)
D = []

**Tiến hành viết một số hàm hỗ trợ cho việc đọc dữ liệu, xử lý và tách từ trong tiếng Việt.**

In [6]:
# Đọc danh sách các từ stopwords trong tiếng Việt
vn_stopwords_file_path = '/content/IRS/data/stopwords/vietnamese-stopwords.txt'
stopwords = []
with open(vn_stopwords_file_path, 'r', encoding='utf-8') as f:
  for line in f:
    line = line.strip()
    stopwords.append(line)

# Viết hàm tiền xử lý và tách từ tiếng Việt
def preprocess(doc):
  # Tiến hành xử lý các lỗi từ/câu, dấu câu, v.v. trong tiếng Việt với hàm text_normalize
  normalized_doc = text_normalize(doc)
  # Tiến hành tách từ
  tokens = word_tokenize(normalized_doc)
  # Lọc stopwords
  non_stopword_tokens = []
  for token in tokens:
    if token not in stopwords:
      non_stopword_tokens.append(token)
  # Tiến hành kết hợp các từ ghép trong tiếng Việt bằng '_'
  combined_tokens = [token.replace(' ', '_') for token in non_stopword_tokens]
  return (normalized_doc, combined_tokens)

# Viết hàm lấy danh sách các văn bản/tài liệu thuộc các chủ đề khác nhau
def fetch_doc_by_topic(topic, limit = 10):
  data_root_dir_path = '/content/IRS/data/vnexpress/{}'.format(topic)
  docs = []
  doc_count = 0
  for file_name in os.listdir(data_root_dir_path):
    if doc_count >= limit:
      break
    file_path = os.path.join(data_root_dir_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
      lines = []
      for line in f:
        line = line.lower().strip()
        lines.append(line)
    doc = " ".join(lines)
    clean_doc = re.sub('\W+',' ', doc)
    (normalized_doc, tokens) = preprocess(clean_doc)
    docs.append((topic, normalized_doc, tokens))
    doc_count+=1
  return docs

**Tiến hành tạo tập dữ liệu thử nghiệm với các tài liệu/văn bản thuộc danh sách chủ đề [topics] đã lựa chọn bên trên**

In [7]:
# Cấu trúc dữ liệu dictionary để lưu thông tin chủ đề-tài liệu, nhằm hỗ trợ cho việc tìm kiếm nhanh
topic_doc_idxes_dict = {}
doc_idx_topic_dict = {}

# Duyệt qua từng chủ đề
doc_idx = 0
for topic in topics:
  current_topic_docs = fetch_doc_by_topic(topic, limit_doc_per_topic)
  topic_doc_idxes_dict[topic] = []
  for (topic, normalized_doc, tokens) in current_topic_docs:
    topic_doc_idxes_dict[topic].append(doc_idx)
    doc_idx_topic_dict[doc_idx] = topic
    doc_idx+=1
  D += current_topic_docs

doc_size = len(D)

print('Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [{}]'.format(doc_size))
for topic in topic_doc_idxes_dict.keys():
  print(' - Chủ đề [{}] có [{}] tài liệu/văn bản.'.format(topic, len(topic_doc_idxes_dict[topic])))

Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [15]
 - Chủ đề [the-thao] có [5] tài liệu/văn bản.
 - Chủ đề [du-lich] có [5] tài liệu/văn bản.
 - Chủ đề [khoa-hoc] có [5] tài liệu/văn bản.


**Tiến hành biến đổi các tài liệu/văn bản trong tập (D) về dạng các TF-IDF vectors - trong bài thực hành này chúng ta sẽ sử dụng thư viện Scikit-Learn (TfidfVectorizer) https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html**

In [8]:
# Khởi tạo đối tượng TfidfVectorizer
vectorizer = TfidfVectorizer()

# Chúng ta sẽ tạo ra một tập danh sách các tài liệu/văn bản dạng list đơn giản để thư viện Scikit-Learn có thể đọc được
sk_docs = []

# Duyệt qua từng tài liệu/văn bản có trong (D)
for (topic, normalized_doc, tokens) in D:
  # Chúng ta sẽ nối các từ/tokens đã được tách để làm thành một văn bản hoàn chỉnh
  text = ' '.join(tokens)
  sk_docs.append(text)

# Tiến hành chuyển đổi các tài liệu/văn bản về dạng các TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(sk_docs)
print(f'Kích thước ma trận tf-idf: [{tfidf_matrix.shape}]')

# Chuyển ma trận tfidf_matrix từ dạng cấu trúc thưa sang dạng đầy đủ để thuận tiện cho việc tính toán
tfidf_matrix = tfidf_matrix.todense().tolist()

Kích thước ma trận tf-idf: [(15, 1843)]


**Tiến hành chuẩn bị tập dữ liệu cho bài toán gom cụm với thuật toán K-means, với đầu vào sẽ là danh sách các tài liệu/văn bản đã được chuyển đổi về dạng các vector TF-IDF**

In [9]:
# Khởi tạo một danh sách chứa thông tin về mã định danh tài liệu (doc_idx) và tf-idf vector tương ứng của nó
docs = []
for doc_idx, doc_tfidf_vector in enumerate(tfidf_matrix):
  docs.append((doc_idx, doc_tfidf_vector))

**Tiến hành cài đặt thuật toán k-means, trong đó điểm trọng tâm (centroid) của một cụm ($c$), ký hiệu $\vec{μ}(c)$ sẽ được xác định như sau:**
# $$\vec{μ}(c)=\frac{1}{|c|}\sum_{\vec{x}∈c}\vec(x)$$

In [10]:
#Function to implement steps given in previous section
class KMeans_Algorithm():
  def __init__(self, docs, k, max_iterations):
        self.docs = np.asarray(docs)
        self.doc_idxes = np.asarray([doc_idx for (doc_idx, doc_vector) in self.docs])
        self.doc_vectors = np.asarray([doc_vector for (doc_idx, doc_vector) in self.docs])
        self.N = len(docs)
        self.k = k
        self.max_iterations = max_iterations
        self.clusters = None
  
  # Hàm chính chạy thuật toán
  def run(self):

    # Lựa chọn ngẫu nhiên vị trí của các điểm trung tâm (centroids) của các cụm bằng việc chọn ngẫu nhiên một số lượng tài liệu/văn bản bằng số (l)
    random_selected_doc_idxes = np.random.choice(len(self.doc_vectors), self.k, replace=False)
    # Dùng (k) các văn bản/tài liệu lựa chọn ngẫu nhiên trên để làm centroids ban đầu
    centroids = self.doc_vectors[random_selected_doc_idxes, :]
     
    # Chúng ta sẽ dùng hàm cdist của thư viện scipy để tính khoảng cách của các tài liệu/văn bản đến các (centroids)
    # Kết quả trả về sẽ là một ma trận khoảng cách có kích thước: n x k - trong đó n: số lượng tài liệu/văn bản và (k) là số lượng cụm.
    #Ví dụ: với (4) tài liệu và (3) cụm, ta sẽ có ma trận khoảng cách như sau
    #  [[17.49285568  0.          8.60232527]
    #   [ 9.2736185   8.60232527  0.        ]
    #   [ 3.         15.58845727  8.06225775]
    #   [ 0.         17.49285568  9.2736185 ]]
    # Ở đây chúng ta sẽ sử dụng khoản cách Euclid
    dists = distance.cdist(self.doc_vectors, centroids ,'euclidean')
     
    # Chúng ta tiến hành xác định cụm của các tài liệu/văn bản dựa trên khoảng cách nhỏ nhất
    # Cấu trúc ở dạng list - với kích thước = số lượng tài liệu/văn bản, ví dụ với k = 3: [2 1 0 0] 
    self.clusters = np.array([np.argmin(dist) for dist in dists])
  

    # Tiến hành cập nhật lại điểm trọng tâm/centroids của các cụm với 1 số lần lặp [max_iterations]
    # Quá trình cập nhật sẽ bao gồm: 
    # BƯỚC 1: tính điểm trọng tâm/centroid mới - dựa trên tính trung bình khoảng cách giữa các tài liệu/văn bản
    # BƯỚC 2: tính lại khoảng cách giữa các tài liệu/văn bản cho các centroids mới 
    # BƯỚC 3: Cập nhật lại thông tin cụm cho các tài liệu/văn bản dựa trên khoảng cách với các centroids mới
    for _ in range(self.max_iterations): 

        # BƯỚC 1: tính điểm trọng tâm/centroid mới
        new_centroids = []
        for cluster_idx in range(self.k):
            # Chúng ta tính khoảng cách trung bình bằng hàm mean(axis=0) - theo hàng và axis=1 là theo cột
            # Ví dụ, ta có: matrix = np.asarray([
            #  [1, 2, 3],
            #  [4, 5, 6],
            #  [6, 8, 9],
            # [10, 11, 12]
            #])
            # matrix.mean(axis=0) -> array([5.25, 6.5 , 7.5 ])
            # matrix.mean(axis=1) -> array([ 2.,  5., 7.66666667, 11.])
            new_centroid_for_cluster_idx = self.doc_vectors[self.clusters==cluster_idx].mean(axis=0) 
            new_centroids.append(new_centroid_for_cluster_idx)
        
        # Cập nhật lại các điểm trọng tâm/centrods
        # Chuẩn hóa lại dữ liệu các centroids từ dạng danh sách: [array([1., 2., 3.]), array([10. , 11. , 13.5]), array([ 5.,  5., 10.])]
        # Thành dạng ma trận numpy nhằm thuận tiện cho việc tính toán: 
        # [[ 1.   2.   3. ]
        #  [10.  11.  13.5]
        #  [ 5.   5.  10. ]]
        centroids = np.vstack(new_centroids)

        # BƯỚC 2: tính lại khoảng cách giữa các tài liệu/văn bản cho các centroids mới
        dists = distance.cdist(self.doc_vectors, centroids ,'euclidean') #Step 2

        # BƯỚC 3: Cập nhật lại thông tin cụm cho các tài liệu/văn bản dựa trên khoảng cách với các centroids mới
        self.clusters = np.array([np.argmin(dist) for dist in dists])

  # Xuất thông tin các cụm sau khi hoàn tất
  def show_cluster_info(self):
    for cluster_idx in range(self.k):
      print("Cụm (cluster) [{}]:".format(cluster_idx))     
      for doc_idx in self.doc_idxes[self.clusters==cluster_idx]:
        print(f" - Chủ đề: [{doc_idx_topic_dict[doc_idx]}]/mã tài liệu: [{doc_idx}]")


**Tiến hành chạy thuật toán K-means với tập dữ liệu thuộc các chủ đề khác trong nhau tập [docs] - chúng ta sẽ lấy giá trị số lượng cụm kỳ vọng bằng với số lượng chủ đề - ví dụ: k = 3**

In [11]:
# Khai báo số lượng cụm kỳ vọng (k)
k = number_of_topics

max_iterations = 100

# Khởi tạo & khai báo các tham số cho mô hình gom cụm HAC 
kmeans_algorithm = KMeans_Algorithm(docs, k, max_iterations)

# Tiến hành chạy thuật toán HAC
print(f'Tiến hành chạy thuật toán KMeans với tập dữ liệu: [{len(docs)}] tài liệu/văn bản - số cụm kỳ vọng (k) = [{k}]...')
kmeans_algorithm.run()

# Hoàn tất - xuất kết quả thông tin các cụm
print('Hoàn tất ! xuất thông tin các cụm...')
kmeans_algorithm.show_cluster_info()

Tiến hành chạy thuật toán KMeans với tập dữ liệu: [15] tài liệu/văn bản - số cụm kỳ vọng (k) = [3]...
Hoàn tất ! xuất thông tin các cụm...
Cụm (cluster) [0]:
 - Chủ đề: [khoa-hoc]/mã tài liệu: [11]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [14]
Cụm (cluster) [1]:
 - Chủ đề: [du-lich]/mã tài liệu: [5]
 - Chủ đề: [du-lich]/mã tài liệu: [6]
 - Chủ đề: [du-lich]/mã tài liệu: [8]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [10]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [12]
 - Chủ đề: [khoa-hoc]/mã tài liệu: [13]
Cụm (cluster) [2]:
 - Chủ đề: [the-thao]/mã tài liệu: [0]
 - Chủ đề: [the-thao]/mã tài liệu: [1]
 - Chủ đề: [the-thao]/mã tài liệu: [2]
 - Chủ đề: [the-thao]/mã tài liệu: [3]
 - Chủ đề: [the-thao]/mã tài liệu: [4]
 - Chủ đề: [du-lich]/mã tài liệu: [7]
 - Chủ đề: [du-lich]/mã tài liệu: [9]


<ipython-input-10-21d572b25428>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.docs = np.asarray(docs)
